# Project Abysima: Generating a Language with Machine Learning

The following notebook will experiment with generating a language using neural networks and generative deep learning.
This is, by no means, a production-ready system, nor is it a complete network; rather, the purpose of this experiment
is to see what is possible with creating a language.

For more information on the process and supporting research, please refer to the Linguistics Paper document found in the
`01 - Areas of Responsibility` directory.

The following source code and datasets are licensed under the Mozilla Public License v2.0. Please refer to the LICENSE
file that came with this repository for more information on what your rights are with usage and modification of this
software. If a LICENSE file is not provided, you can obtain a copy at https://www.mozilla.org/en-US/MPL/2.0/.

## Part I: Initial Setup

This notebook will utilize the TensorFlow and Keras libraries to create the generative neural networks for this project,
as well as some other Python libraries to parse the data from CSV files or other file formats.

In [ ]:
# Import the Keras packages we will use to create the neural network models, and to parse the dataset.
from tensorflow import keras

## Part II: Dataset Parsing and Encoding

Before the dataset can be thrown into the neural network, we first need to sanitize and prepare the dataset for use with
the network. This will also include creating training, validation, and testing sets to ensure accuracy while mitigating
overfitting.

In [ ]:
# TODO: Implement this code block